In [3]:
using Plots;
using DifferentialEquations;
using LinearAlgebra;

In [2]:
include("Solvers.jl")
include("PlotFunctions.jl")
include("UsefulFunctions.jl")
include("PointGenerators.jl")

PointsOnSphere (generic function with 2 methods)

In [74]:
θ, β = 10, 8/3

Δt = 0.1
N = 500

Δμ = 0.1

0.1

In [75]:
maxμ = θ*(θ+β+3)/(θ-β-1)
minμ = 1 + Δμ

1.1

In [76]:
for solver = [DTM RK4 EulerForward EulerCromer RadauIIA5 Rodas5]
    i = 0
    j = 0
    vec = []
    for μ = minμ:Δμ:maxμ
        i = i + 1
        p = [θ, μ, β]
        val = sqrt(β*(μ-1))
        fixpoints = [[0, 0, 0], [val, val, μ-1], [-val, -val, μ-1]]
        M = PointsSolutions(p,Δt,N,fixpoints,solver, 100)
        for k = 1:3
            
            if M[k][:,end] != fixpoints[k]
                j = j+ 1
                nor = norm(M[k][:,end]-fixpoints[k])
                push!(vec, nor)
            end
        end
        
    end
    l = length(vec)
    print("l=", l, " i=", i, "\n")
    if l > 0
        print(solver, " failed ", j, " times.", " max=", maximum(vec), " min=", minimum(vec), " avg=", sum(vec)/length(vec), "\n")
    else
        print(solver, " failed ", j, " times.", " max=", 0, " min=", 0, " avg=", 0, "\n")
    end
end

l=0i=711
DTM failed 0 times. max=0 min=0 avg=0


l=0i=711
RK4 failed 0 times. max=0 min=0 avg=0


l=0i=711
EulerForward failed 0 times. max=0 min=0 avg=0


l=0i=711
EulerCromer failed 0 times. max=0 min=0 avg=0


l=0i=711
RadauIIA5 failed 0 times. max=0 min=0 avg=0


l=0i=711
Rodas5 failed 0 times. max=0 min=0 avg=0
